In [1]:
import pandas as pd
import os
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

In [2]:
# StandarizedRunway32 | PCARunway32 | StandarizedRunway18  | PCARunway18
dataset = 'PCARunway32.csv'

In [3]:
# Path con los datasets iniciales
#os.chdir('D:/Documents/UPM/Actual/TFG/LEMD/DataSets')
#os.chdir('C:/Users/aalfaro/Documents/Meteo/TFG/LEMD/DataSets')

os.chdir('/home/crida/notebooks/Meteo TFG Adrian/Datasets')

#pd.options.display.max_rows = 200
#pd.options.display.max_columns = 50

#pd.set_option('display.float_format', lambda x: '%.10f' % x)

In [4]:
df = pd.read_csv(dataset, header = 0, decimal = ',', sep = ';')
df

,threshold_separation,0,1,2,3,4,5,6,7,8,9
0,4.549984,-1.804886,1.133685,-2.137942,1.302597,-0.317661,-0.360252,1.534981,-1.198028,1.324067,-1.294057
1,7.301271,-1.852217,1.157554,-1.808590,1.195718,-0.195919,-0.165585,0.777213,-0.482541,-0.077994,-0.278638
2,11.165620,-1.819400,1.056051,-2.194035,1.311567,0.060222,0.101954,0.145174,0.003649,-0.085783,0.108939
3,4.485055,-1.846906,1.134366,-1.932873,1.226929,-0.117933,-0.434605,1.081362,2.334074,0.520167,-1.226675
4,4.462536,-1.787726,0.975369,-2.544367,1.415456,0.243610,0.284158,-0.239561,0.245998,-0.065172,0.274369
...,...,...,...,...,...,...,...,...,...,...,...
55222,6.137123,-1.831997,0.898585,-1.801295,1.094508,-0.074166,0.010041,0.618513,-0.681769,-0.339695,-0.470735
55223,6.956980,-1.930480,1.170384,-0.456243,0.749209,-0.843761,-0.780117,2.316297,-1.106373,3.456839,-1.539166
55224,5.514888,-1.891279,1.027651,-0.999592,0.900539,-0.428695,-0.333982,1.225570,-0.390831,2.479556,-0.743425
55225,4.829517,-1.848150,0.957218,-1.451756,1.025280,-0.311486,-0.245155,1.191013,-0.643097,2.161459,-0.961724


In [5]:
X = df.drop(['threshold_separation'], axis=1)
y = df['threshold_separation']

In [6]:
# Regresion lineal
lr = LinearRegression()
params = {
    'fit_intercept': [True]
}

model = GridSearchCV(
    estimator = lr,
    param_grid = params,
    scoring = ['neg_root_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_squared_log_error', 'neg_mean_poisson_deviance'], 
    refit = 'neg_root_mean_squared_error',
    cv = 5,
    n_jobs = 5
)

model.fit(X, y)

GridSearchCV(cv=5, estimator=LinearRegression(), n_jobs=5,
             param_grid={'fit_intercept': [True]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2', 'explained_variance',
                      'neg_mean_squared_log_error',
                      'neg_mean_poisson_deviance'])

In [7]:
a = pd.concat([pd.DataFrame(model.cv_results_["params"])
              ,pd.DataFrame(model.cv_results_['rank_test_neg_root_mean_squared_error'], columns=['Rank'])
              ,pd.DataFrame(model.cv_results_['mean_fit_time'], columns=['Fit Time'])
              ,pd.DataFrame(model.cv_results_['mean_score_time'], columns=['Score Time'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_root_mean_squared_error'], columns=['RMSE'])
              ,pd.DataFrame(model.cv_results_['mean_test_r2'], columns=['R2'])
              ,pd.DataFrame(model.cv_results_['mean_test_explained_variance'], columns=['EV'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_squared_log_error'], columns=['MSLE'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_poisson_deviance'], columns=['MPD'])]
              ,axis=1)

a = a.sort_values(by = ['Rank', 'Score Time'], ascending=True)
a.to_csv('Results/32PCA-LR.csv', decimal = ',', sep = ';', index = False)

a

,fit_intercept,Rank,Fit Time,Score Time,RMSE,R2,EV,MSLE,MPD
0,True,1,0.012483,0.003199,-1.820573,0.034542,0.038517,-0.05824,-0.508582


In [8]:
# KNN
knn = KNeighborsRegressor()
params = {
    'n_neighbors': [10, 20, 30, 40],
    'weights': ['distance', 'uniform'],
    'metric': ['minkowski'], 
    'p': [2]
}

model = GridSearchCV(
    estimator = knn,
    param_grid = params,
    scoring = ['neg_root_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_squared_log_error', 'neg_mean_poisson_deviance'], 
    refit = 'neg_root_mean_squared_error',
    cv = 5,
    n_jobs = 5
)

model.fit(X, y)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=5,
             param_grid={'metric': ['minkowski'],
                         'n_neighbors': [10, 20, 30, 40], 'p': [2],
                         'weights': ['distance', 'uniform']},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2', 'explained_variance',
                      'neg_mean_squared_log_error',
                      'neg_mean_poisson_deviance'])

In [9]:
b = pd.concat([pd.DataFrame(model.cv_results_["params"])
              ,pd.DataFrame(model.cv_results_['rank_test_neg_root_mean_squared_error'], columns=['Rank'])
              ,pd.DataFrame(model.cv_results_['mean_fit_time'], columns=['Fit Time'])
              ,pd.DataFrame(model.cv_results_['mean_score_time'], columns=['Score Time'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_root_mean_squared_error'], columns=['RMSE'])
              ,pd.DataFrame(model.cv_results_['mean_test_r2'], columns=['R2'])
              ,pd.DataFrame(model.cv_results_['mean_test_explained_variance'], columns=['EV'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_squared_log_error'], columns=['MSLE'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_poisson_deviance'], columns=['MPD'])]
              ,axis=1)

b = b.sort_values(by = ['Rank', 'Score Time'], ascending=True)
b.to_csv('Results/32PCA-KNN.csv', decimal = ',', sep = ';', index = False)

b

,metric,n_neighbors,p,weights,Rank,Fit Time,Score Time,RMSE,R2,EV,MSLE,MPD
6,minkowski,40,2,distance,1,0.060083,1.142146,-1.801026,0.054943,0.061200,-0.057356,-0.496560
7,minkowski,40,2,uniform,2,0.054880,0.961582,-1.803270,0.052611,0.059022,-0.057530,-0.497871
4,minkowski,30,2,distance,3,0.059366,1.042872,-1.805747,0.049950,0.055992,-0.057660,-0.499362
5,minkowski,30,2,uniform,4,0.054794,0.939944,-1.807488,0.048139,0.054306,-0.057799,-0.500399
2,minkowski,20,2,distance,5,0.050490,0.702273,-1.816772,0.038247,0.044054,-0.058383,-0.505809
3,minkowski,20,2,uniform,6,0.058894,0.893020,-1.817846,0.037129,0.043036,-0.058474,-0.506471
1,minkowski,10,2,uniform,7,0.061322,0.702721,-1.854485,-0.002165,0.003458,-0.060983,-0.527864
0,minkowski,10,2,distance,8,0.075009,0.704130,-1.854813,-0.002532,0.003012,-0.060981,-0.527986


In [10]:
# Random Forest Regression
rfr = RandomForestRegressor(random_state = 1)
params = {
    'n_estimators': [100, 150, 200],
    'criterion': ['mse'],
    'max_depth': [None, 10, 12],
    'max_leaf_nodes': [None],
    'bootstrap': [True],
    'max_features': [0.5],
    'min_samples_split': [10, 15], 
    'min_samples_leaf': [5, 10]
}

model = GridSearchCV(
    estimator = rfr,
    param_grid = params,
    scoring = ['neg_root_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_squared_log_error', 'neg_mean_poisson_deviance'], 
    refit = 'neg_root_mean_squared_error',
    cv = 5,
    n_jobs = 5
)

model.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=1), n_jobs=5,
             param_grid={'bootstrap': [True], 'criterion': ['mse'],
                         'max_depth': [None, 10, 12], 'max_features': [0.5],
                         'max_leaf_nodes': [None], 'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 15],
                         'n_estimators': [100, 150, 200]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2', 'explained_variance',
                      'neg_mean_squared_log_error',
                      'neg_mean_poisson_deviance'])

In [11]:
c = pd.concat([pd.DataFrame(model.cv_results_["params"])
              ,pd.DataFrame(model.cv_results_['rank_test_neg_root_mean_squared_error'], columns=['Rank'])
              ,pd.DataFrame(model.cv_results_['mean_fit_time'], columns=['Fit Time'])
              ,pd.DataFrame(model.cv_results_['mean_score_time'], columns=['Score Time'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_root_mean_squared_error'], columns=['RMSE'])
              ,pd.DataFrame(model.cv_results_['mean_test_r2'], columns=['R2'])
              ,pd.DataFrame(model.cv_results_['mean_test_explained_variance'], columns=['EV'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_squared_log_error'], columns=['MSLE'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_poisson_deviance'], columns=['MPD'])]
              ,axis=1)

c = c.sort_values(by = ['Rank', 'Score Time'], ascending=True)
c.to_csv('Results/32PCA-RFR.csv', decimal = ',', sep = ';', index = False)

c

,bootstrap,criterion,max_depth,max_features,max_leaf_nodes,min_samples_leaf,min_samples_split,n_estimators,Rank,Fit Time,Score Time,RMSE,R2,EV,MSLE,MPD
26,True,mse,12.0,0.5,None,5,10,200,1,25.485872,0.148761,-1.763574,0.093913,0.099990,-0.054085,-0.473709
8,True,mse,NaN,0.5,None,10,10,200,2,34.529852,0.230091,-1.763602,0.093768,0.099907,-0.054132,-0.473710
11,True,mse,NaN,0.5,None,10,15,200,2,34.538122,0.230283,-1.763602,0.093768,0.099907,-0.054132,-0.473710
35,True,mse,12.0,0.5,None,10,15,200,4,23.956518,0.138346,-1.763734,0.093753,0.099823,-0.054052,-0.473772
32,True,mse,12.0,0.5,None,10,10,200,4,27.131846,0.153970,-1.763734,0.093753,0.099823,-0.054052,-0.473772
29,True,mse,12.0,0.5,None,5,15,200,6,25.406908,0.147223,-1.763754,0.093726,0.099781,-0.054115,-0.473830
31,True,mse,12.0,0.5,None,10,10,150,7,20.351524,0.116195,-1.763834,0.093649,0.099715,-0.054061,-0.473826
34,True,mse,12.0,0.5,None,10,15,150,7,20.360336,0.116298,-1.763834,0.093649,0.099715,-0.054061,-0.473826
25,True,mse,12.0,0.5,None,5,10,150,9,20.694226,0.117802,-1.763986,0.093486,0.099601,-0.054117,-0.473953
28,True,mse,12.0,0.5,None,5,15,150,10,20.623749,0.116664,-1.764030,0.093442,0.099545,-0.054139,-0.474013


In [12]:
# SVR
svr = SVR()
params = {
    'kernel': ['rbf'],
    'gamma': [0.001, 0.01, 0.1, 1],
    'C': [0.01, 0.1, 1.0, 10.0]
}

model = GridSearchCV(
    estimator = svr,
    param_grid = params,
    scoring = ['neg_root_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_squared_log_error', 'neg_mean_poisson_deviance'], 
    refit = 'neg_root_mean_squared_error',
    cv = 3,
    n_jobs = 3
)

model.fit(X, y)

GridSearchCV(cv=3, estimator=SVR(), n_jobs=3,
             param_grid={'C': [0.01, 0.1, 1.0, 10.0],
                         'gamma': [0.001, 0.01, 0.1, 1], 'kernel': ['rbf']},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2', 'explained_variance',
                      'neg_mean_squared_log_error',
                      'neg_mean_poisson_deviance'])

In [13]:
d = pd.concat([pd.DataFrame(model.cv_results_["params"])
              ,pd.DataFrame(model.cv_results_['rank_test_neg_root_mean_squared_error'], columns=['Rank'])
              ,pd.DataFrame(model.cv_results_['mean_fit_time'], columns=['Fit Time'])
              ,pd.DataFrame(model.cv_results_['mean_score_time'], columns=['Score Time'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_root_mean_squared_error'], columns=['RMSE'])
              ,pd.DataFrame(model.cv_results_['mean_test_r2'], columns=['R2'])
              ,pd.DataFrame(model.cv_results_['mean_test_explained_variance'], columns=['EV'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_squared_log_error'], columns=['MSLE'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_poisson_deviance'], columns=['MPD'])]
              ,axis=1)

d = d.sort_values(by = ['Rank', 'Score Time'], ascending=True)
d.to_csv('Results/32PCA-SVR.csv', decimal = ',', sep = ';', index = False)

d

,C,gamma,kernel,Rank,Fit Time,Score Time,RMSE,R2,EV,MSLE,MPD
10,1.00,0.100,rbf,1,67.507461,25.672180,-1.835110,0.020791,0.060236,-0.057046,-0.519502
6,0.10,0.100,rbf,2,65.526377,25.772233,-1.841617,0.013769,0.056806,-0.057100,-0.522463
7,0.10,1.000,rbf,3,65.815510,25.655759,-1.842909,0.012645,0.056603,-0.057280,-0.522202
13,10.00,0.010,rbf,4,71.200921,25.755894,-1.847246,0.007619,0.045912,-0.057814,-0.527251
11,1.00,1.000,rbf,5,115.997715,25.587352,-1.851824,0.003148,0.041524,-0.058556,-0.530163
8,1.00,0.001,rbf,6,83.994366,25.796857,-1.855783,-0.001505,0.039666,-0.057957,-0.531118
2,0.01,0.100,rbf,7,65.976223,25.731250,-1.855802,-0.001545,0.042745,-0.057984,-0.530495
9,1.00,0.010,rbf,8,67.357117,25.768408,-1.855875,-0.001692,0.039945,-0.058065,-0.531744
4,0.10,0.001,rbf,9,83.682698,25.835834,-1.856095,-0.001841,0.039959,-0.057981,-0.530851
5,0.10,0.010,rbf,10,66.926130,25.796932,-1.857296,-0.003166,0.039313,-0.058046,-0.532117
